In [1]:
# Import Azure OpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from sklearn.feature_extraction.text import CountVectorizer

from dotenv import load_dotenv

from collections import Counter
from bertopic import BERTopic
from nltk.corpus import stopwords
import numpy as np

%load_ext autoreload
%autoreload 2

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#import from https://towardsdatascience.com/topic-modelling-using-chatgpt-api-8775b0891d16
import tiktoken 
import pandas as pd
gpt4_enc = tiktoken.encoding_for_model("gpt-4")

def get_tokens(enc, text):
    return list(map(lambda x: enc.decode_single_token_bytes(x).decode('utf-8'), 
                  enc.encode(text)))

In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
load_dotenv() 

# setting up APIKey to access ChatGPT API
openai.api_key  = os.environ['OPENAI_API_KEY'] 


# simple function that return just model response
def get_model_response(messages, 
                       model = 'gpt-3.5-turbo', 
                       temperature = 0, 
                       max_tokens = 1000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )

    return response.choices[0].message['content']


# we can also return token counts
def get_model_response_with_token_counts(messages, 
                                   model = 'gpt-3.5-turbo', 
                                   temperature = 0, 
                                   max_tokens = 1000):
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message['content']
    
    tokens_count = {
      'prompt_tokens':response['usage']['prompt_tokens'],
      'completion_tokens':response['usage']['completion_tokens'],
      'total_tokens':response['usage']['total_tokens'],
    }

    return content, tokens_count

In [5]:
from bertopic.representation import OpenAI

Import Data from CSV

In [6]:
path_csv= "data/"

df = pd.read_csv(path_csv+'transcriptions.csv')
# Filter out videos with the same video_id
df = df.drop_duplicates(subset=['video_id'])


In [7]:
docs_classifier = [
   str(df["video_description"][a]) + "\n\n" +  str(df["suggested_words"][a]) +"\n\n" +  str(df["german_transcript"][a])
    for a in df.index
]

docs = [str(df["german_transcript"][a])
    for a in df.index
]

print(len(docs))

790


In [8]:
summarization_prompt = """
Ich habe ein Thema, das durch die folgenden Stichwörter beschrieben wird: [KEYWORDS]
In diesem Thema sind die folgenden Dokumente eine kleine, aber repräsentative Untermenge aller Dokumente im Thema:
[DOCUMENTS]

Basierend auf den oben genannten Informationen geben Sie bitte eine Beschreibung dieses Themas in einer Aussage im folgenden Format ab:
Thema:  <description>
"""

In [54]:
representation_model= AzureChatOpenAI(
    openai_api_version="2024-02-01",
    azure_deployment="gpt-35-turbo-16k", chat=True, prompt=summarization_prompt, 
                              nr_docs=10, delay_in_seconds=3
)

embeddings_client = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-3-small-eastus",
    openai_api_version="2024-02-01",
)

In [23]:
german_stop_words= open("data/clustering/german_stopwords_full.txt").read().split()[53:]

In [55]:
embeddings = np.load(path_csv+"/clustering/transcriptions.npy")
topic_model = BERTopic() #nr_topics = 50, embedding_model=embeddings_client
topics, ini_probs = topic_model.fit_transform(docs, embeddings)


In [57]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,34,-1_sie_fr_einfach_vaterland,"[sie, fr, einfach, vaterland, hnden, unseren, ...",[einfach 1 bisschen schneller rausgehen. bleib...
1,0,603,0_die_und_der_das,"[die, und, der, das, ich, ist, in, nicht, wir,...","[es ist eiskalter Kaffee, den sie jetzt wieder..."
2,1,125,1_die_und_das_in,"[die, und, das, in, ja, der, auch, hier, mensc...",[tun Deutschland hat mit den Entscheidung für ...
3,2,14,2_gestrichen_nicht_werden_prsent,"[gestrichen, nicht, werden, prsent, wir, ersat...",[wenn diese beiden Maßnahmen nicht gestrichen ...
4,3,14,3_sie_unseris_wer_gelacht,"[sie, unseris, wer, gelacht, hab, kaputt, gese...",[wer sind sie eigentlich? hab sie noch nie ges...


In [52]:
repr_docs, _, _ = topic_model._extract_representative_docs(c_tf_idf=topic_model.c_tf_idf_,
                                                          documents=pd.DataFrame(docs),
                                                          topics=topic_model.topic_representations_ ,
                                                          nr_repr_docs=50)

KeyError: 'Topic'

In [50]:
len(repr_docs)

8

In [40]:
delimiter = '####'
system_message = '''Du bist ein Experte im Schreiben. Du hilfst mir, einen gemeinsamen Titel für mehrere ähnliche TikTok-Beiträge zu schreiben. Der Titel sollte das gemeinsame Narrativ bestmöglichst beschreiben. Beispiele sind: Anti-Einwanderung: Grenzen schließen, nationale Identität schützen. Kritik am Establishment: Gegner des politischen Establishments, Alternativpartei. Kulturelle Identität: Betonung der deutschen Kultur, Traditionen bewahren. Anti-EU: Kritik an der EU, nationale Souveränität betonen. Forderung nach einer harte Hand gegen Kriminalität und Terrorismus. Hetzerischer Angriff und Kritik an Grünen ohne konstruktive Gegenvorschläge.
'''
user_message = f'''
Untern findest du repräsentative TikTok Videos {delimiter}. 
Bitte identifiziere die Hauptthemen, die in diesen Kommentaren erwähnt wurden.

Gebe eine Liste von 10-20 Themen zurück.
Die Ausgabe ist eine JSON-Liste im folgenden Format:
[
    {{"topic_name": "<topic1>", "topic_description": "<topic_description1>"}}, 
    {{"topic_name": "<topic2>", "topic_description": "<topic_description2>"}},
    ...
]

TikTok Videos:
{delimiter}
{delimiter.join(repr_docs)}
{delimiter}
'''


messages =  [  
        {'role':'system', 
         'content': system_message},    
        {'role':'user', 
         'content': f"{user_message}"},  
] 

TypeError: sequence item 0: expected str instance, int found